# Huffman Coding

In [30]:
from pandas import DataFrame

import numpy as np

message = r'In computer science and information theory, a Huffman code is a particular type of optimal prefix code that is commonly used for lossless data compression. The process of finding or using such a code proceeds by means of Huffman coding, an algorithm developed by David A. Huffman while he was a Sc.D. student at MIT, and published in the 1952 paper "A Method for the Construction of Minimum-Redundancy Codes".'

print(DataFrame(list(message), columns=['message']).groupby('message').size().sort_values().index.to_list())

['x', '-', '1', '2', '5', '9', 'S', 'R', 'C', '"', 'w', 'A', 'v', 'D', 'T', 'I', 'b', 'H', ',', 'M', 'g', '.', 'y', 'l', 'h', 'p', 'u', 'm', 'r', 'f', 'c', 't', 'd', 'i', 's', 'a', 'n', 'o', 'e', ' ']
